# Augmented VR Module

In [1]:
import inspect
from viper_toolkit import Dissect
from scripts import augmented_vr
from scripts.augmented_vr import AugmentedVRModule

In [2]:
source = inspect.getsource(AugmentedVRModule)
print (source)

class AugmentedVRModule(object):
    
    def __init__(self):
        self.name = NameManager()
        #self.dynamic_updates()
        self.pose_estimations = InferenceResults()
        self.setup_parameters()
        self.setup_ros()
        self.setup_scene_segmentation()
        self.loop()

    def setup_parameters(self):
        # Adds all of our parameters to an updatable list. If ROS cannot
        # find the parameter, it will use the default provided.
        self.parameters = ParameterManager()
        
        self.parameters.add(Parameter("width", "/inland_ir_cam/width", default = 800, dynamic = True))
        self.parameters.add(Parameter("height", "/inland_ir_cam/height", 600, True))
        self.parameters.add(Parameter("updates", f"{self.name.name}/dynamic", True, True))
        self.parameters.add(Parameter("image_output", f"{self.name.name}/image_output", True, True))
        self.parameters.add(Parameter("alpha", rospy.search_param('alpha'), 0.3, True))
        self

The Scene Segmentation Module is a ROS Package Module which is comprised of:

* the SceneSegmentationModule Class Object
* the SceneSegmentationModel processing library
* the RoadSegmentation ADAS Model and pretrained weights.


## Class Definition

### Initializing the SceneSegmentationModule Class

The class loads a NameManager, the InferenceResults() message type, and the setup functions upon initalization

In [4]:
Dissect("__init__", SceneSegmentationModule)

class SceneSegmentationModule(object):

    def __init__(self):
        self.name = NameManager(abv="SEG")
        self.inference = InferenceResults()
        self.setup_ros()
        #self.setup_model()
        self.loop()



### Setup Paramters
We initalize several parameters with the parameter server, including the rate at which we would like the node to run, as well as if we would like to be able to dynamically update this node. This will allow us to change its rate once it is already running to optimize its performance with the other nodes.

In [6]:
Dissect("setup_parameters", SceneSegmentationModule)

class SceneSegmentationModule(object):

    def setup_parameters(self):
        self.parameters = ParameterManager()
        self.parameters.add(Parameter(
            name="rate", 
            target=f"{self.name.name}/rate", 
            default=50, 
            dynamic=True
            ))
        self.parameters.add(Parameter(
            name="updates", 
            target=f"{self.name.name}/dynamic", 
            default=True, 
            dynamic=True
            ))



#### Setting up ROS and the Model Server
 We setup the module and register it with the ROS network. We also setup the **model_server/modeling_service** ServiceProxy which we will use to request new Models. The request message 'ModelRequest' takes the name of a model (SEG is the name of the Segmentation model) and returns an Inference.

In [11]:
Dissect("setup_ros", SceneSegmentationModule)
Dissect("setup_model_server", SceneSegmentationModule)

class SceneSegmentationModule(object):

    def setup_ros(self):
        self.setup_parameters()
        rospy.init_node('scene_segmentation', log_level=rospy.DEBUG)
        self.setup_model_server()
        self.image_server = ImageServerClient(self.name.abv)

class SceneSegmentationModule(object):

    def setup_model_server(self):
        
        self.model_request = rospy.ServiceProxy(
            'model_server/modeling_service',
            ModelRequest,
            )

        self.pub = rospy.Publisher(
            'model_output/scene_segmentation/segmentation_mask',
            InferenceResults,
            queue_size=1
            )



### ImageServerClient
We also configure the Image Server client, which allows this module to communicate with the image server. It does this much in the same way that a Publisher publishes messages... when the module is ready for a new image it broadcasts that it is waiting for an image. The image server is a subscriber of these broadcasts, and once it hears this message it will provide an image to the model server.

The reason we have chosen this method rather than providing a service is that a service call blocks the kernel from processing more code until the call has completed and a response has been received, whereas a publisher will simply continue to broadcast a message, regardless if it is received or not. This ensures that processes are not tied up in a service call. 

Once the Image Server hears that the client is in need of an image it begins the process and communicates to all nodes after it has updated the model server with the new image. It then changes these nodes "waiting" status to false via the ImageServerClient.callback. This ensures that while the module is engaging the model server the image server does not wastefully deliver images which no module will use, potentially reducing bandwidth by nearly 87% (assuming that an request is made twice a second, and the image server is reciving images at the rate of 16 FPS; since only two of those frames out of 16 can be used, only two will be delivered to the model server which would otherwise receive all 16!).

In [16]:
from image_server import ImageServerClient
print (inspect.getsource(ImageServerClient))

class ImageServerClient(object):
    def __init__(self, name: str):
        self._name = name
        self.waiting = Bool()
        self.remote_status = Bool()
        self.setup_publisher()
        self.setup_subscriber()
        
    def setup_publisher(self):
        self.pub = rospy.Publisher(
            f'image_request/{self._name}',
            Bool,
            queue_size=1
            )
        rospy.loginfo(
            f'[{self._name}] Image server request services online.'
            )
            
    def update(self, state: bool):
        self.waiting.data = state
        self.pub.publish(self.waiting)
   
    def status(self, entity = "server"):
        if entity == "server":
            return self.remote_status.data
        else:
            return self.waiting.data
    
    def refresh(self, state: bool):
        self.waiting.data = state
        
    def setup_subscriber(self):
        rospy.Subscriber(
            'image_server/status',
            Bool,
          

### Action Loop
When the SceneSegmentationModule is ready to perform inference it contacts the model server and provides its modeling code "SEG". The model server then computes the inference, and returns an inference message.

Inference Messages contain two variables:
* int32[]     structure
* float32[]   inferences

The Vision Processing Unit computes the segmentation of the image, and produces an array of variable length indicating the class of each pixel in the image. All ROS messages must be compatable between Python and C++, and Numpy ndarrays are not supported. In order to serialize these variable length inference results, I decided to flatten the numpy darrays into one dimensional vectors. 

The structure variable is a list of 32 bit unsigned integers which indicate the original shape of the arrays produced by the Vision Processing Unit.The inference variable is a list of 32 bit floats. 

The node can reshape the inference vector into the original arrays by using the structure vector as the original shape parameters.

In [9]:
Dissect("action_loop", SceneSegmentationModule)

class SceneSegmentationModule(object):

    def action_loop(self):
        self.timer.lap("Wait for Service")
        rospy.wait_for_service(
            'model_server/modeling_service'
            )
        self.timer.time("Wait for Service")
        try:
            self.timer.lap("Modeling Server")
            rospy.logdebug(f"[{self.name.abv}] Service Requested.")
            req = String()
            req.data = "SEG" #self.name.abv
            response = self.model_request(req)
            self.timer.time("Modeling Server", name = "MOD")
            self.inference = response.results
            rospy.logdebug(
                f"[{self.name.abv}] Mask shape received: {self.inference.structure}"
                )
            return self.inference

        except rospy.ServiceException as e:
            rospy.logerr(f"[{self.name.abv}] Service call failed: {e}")



### The Main Loop
The complete process is as follows:
1. the Node checks for any updates to its parameters and begins logging its time.
2. It then checks to see if there are any new images waiting for it at the model server.
3. If there are new images, it makes a model request.Prior to making the request however it takes down its own "waiting for image" since it cannot simultaniously make a request and use these images.
4. Following sucessfully receiving images it broadcasts the results to its subscribers. It does not update its request durring this time since while it is performing this action it cannot use any new images.
5. After publishing its inference results it updates its flag to reflect that it is ready for a new image which will be delivered by the time the loop repetes itself and it requests a new inference from the model server. 

In [10]:
Dissect("loop", SceneSegmentationModule)

class SceneSegmentationModule(object):

    def loop(self):
        rate = rospy.Rate(self.parameters.rate)
        while not rospy.is_shutdown():
            if self.parameters.updates == True:
                self.parameters.update()
                
            # Setup timer. Note that this resets on every iteration. 
            # Future development we may want to move this so that we can 
            # compute averages.
            
            self.timer = ProcessTimer(abv = self.name.abv)
            
            # Check to see if the server has released a new image and 
            # if so, start new modeling request
            
            status = self.image_server.status("server")
            rospy.logdebug(
                f"[{self.name.abv}] Remote Image Server Status is: {status}"
                )
            if status == True: 
                # Takes down image request flag while performing 
                # inference since inference time is not negligable
          